In [1]:
%load_ext autoreload
%env SPARK_HOME=/usr/hdp/current/spark2-client

import findspark
findspark.init()

print('findspark initialized ...')

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col, column, max, min

spark = SparkSession.builder.appName('mlonspark')\
    .config('spark.executor.instances', '6')\
    .config('spark.jars', '/opt/dev/target/ml-on-spark-1.0.jar')\
    .getOrCreate()

print('pyspark ready ...')

env: SPARK_HOME=/usr/hdp/current/spark2-client
findspark initialized ...
pyspark ready ...


In [2]:
#rawData = spark.read.format('text').load("/data/lastfm/userid-timestamp-artid-artname-traid-traname.tsv").rdd
#rawData = rawData.repartition(6)
#rawData.getNumPartitions()
#rawData.toDF().write.format("text").mode("overwrite").save("/data/lastfm/lastfm-repartitioned.tst")

In [3]:
rawData = spark.read.format('text').load("/data/lastfm/lastfm-repartitioned.tst").rdd
rawData.getNumPartitions()
#rawData.count()

20

In [4]:
def parseLine(line) :
    splits = line.value.split("\t")
    return (splits[0], splits[1], splits[2])
    
parsed = rawData.map(parseLine)
parsed.first()
#parsed.count()

('user_000044', '2008-01-20T10:37:31Z', 'eace2373-31c8-4aba-9a5c-7bce22dd140a')

In [11]:
users = parsed.map(lambda x: (x[0], x))
users = users.groupByKey().map(lambda x: x[0]).zipWithIndex()
users.toDF().write.mode("overwrite").save("/data/lastfm/users.parquet")

In [14]:
usersDF = spark.read.load("/data/lastfm/users.parquet")
usersDF.count()

992

In [12]:
interprets = parsed.map(lambda x: (x[2], x))
interprets = interprets.groupByKey().map(lambda x: x[0]).zipWithIndex()
interprets.toDF().write.mode("overwrite").save("/data/lastfm/interprets.parquet")

In [13]:
interpretsDF = spark.read.load("/data/lastfm/interprets.parquet")
interpretsDF.count()

107398